In [1]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None


In [2]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.06)

In [3]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


520.8235557079315

In [4]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1540.7609683932544, 0.007703804841966272, 184)

In [5]:
output['history']

,impressions,clicks,lifetime,p
153,18970.0,4199.0,18003.025431,0.220134
162,228.0,26.0,1537.166719,0.113780
172,163.0,32.0,19648.592394,0.219968
173,170.0,18.0,12771.474990,0.122694
180,26.0,3.0,4655.819793,0.020061
182,6.0,0.0,889.624649,0.004621
183,1.0,0.0,15187.163761,0.073886


Для реализации я выбрала ucb.

In [6]:
class UCB:
    def __init__(self, C):                    
        self.C = C
        self.timer = 0        
        
    def __call__(self, history: pd.DataFrame):
        self.timer += 1
        ctr = history['clicks'] / (history['impressions'] + 10)
        uncert = self.C * np.sqrt(np.log(self.timer) / (history['impressions'] + 10))
        n = np.argmax(ctr + uncert)
        return history.index[n]

Проведем подбор коэффициента.

In [9]:
for C in [0.2, 0.4, 0.6, 0.8, 1., 2, 5, 10, 0.05]:
    print("C = ", C)
    start = time.time()
    output = simulation(UCB(C=C), n=200000, seed=seed)
    end = time.time()
    end - start
    print(output['regret'], output['regret']/output['rounds'],  output['total_banners'])

C =  0.2
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
163.08533688898632 0.0008154266844449316 184
C =  0.4
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have 

Лучше всего получилось с коэффициентом 0.2, поисследуем немного вокруг.

In [10]:
for C in [0.25, 0.15]:
    print("C = ", C)
    start = time.time()
    output = simulation(UCB(C=C), n=200000, seed=seed)
    end = time.time()
    end - start
    print(output['regret'], output['regret']/output['rounds'],  output['total_banners'])

C =  0.25
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
263.47118188702103 0.0013173559094351052 184
C =  0.15
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions hav

Оставим коэффициент 0.15 как лучший. С ним модель работает лучше, чем бейзлайн.

In [11]:
output['history']

,impressions,clicks,lifetime,p
153,19856.0,4255.0,18003.025431,0.220134
162,1.0,0.0,1537.166719,0.113780
172,1.0,0.0,19648.592394,0.219968
173,1.0,0.0,12771.474990,0.122694
180,0.0,0.0,4655.819793,0.020061
182,0.0,0.0,889.624649,0.004621
183,0.0,0.0,15187.163761,0.073886
